### Import packages

In [0]:
import requests
import urllib
import pandas as pd
import os
from bs4 import BeautifulSoup
import re

### Download HTML pages

In [0]:
def saveashtml(s,file_name):
    with open(file_name,'w') as f:
        f.write(s)
        f.close()

In [0]:
headers = {'User-Agent': 'whatever'}

start_year = 1950
end_year = 1995
url = "http://www.liiofindia.org/in/cases/cen/INSC/"
path = "Web Pages/"

for curr_year in range(start_year,end_year+1,1):
    flag = True
    i = 1
    while flag :
        url_new = url + str(curr_year) + "/" + str(i) + ".html"
        file_name = path + str(curr_year) + "_" + str(i) + ".html"
        result = requests.get(url_new,headers = headers)
        if result.status_code == int(404):
            flag = False
            i = 1
        else:
            saveashtml(result.text,file_name)
            i = i+1

### Data Scraping from HTML pages

In [0]:
# Extract to data frame from HTML pages
def extract2df(soup,filename):
    head_str = ""
    civil_str = ""
    
    full_flag = False
    summary_flag = False
    
    # Extract for Summary
    sum_start = "HEADNOTE:".lower()
    sum_end = "CIVIL APPELLATE JURISDICTION:".lower()
    
    # Extract for Full text
    full_start = sum_end
    full_end = "LIIofIndia:".lower()
    
    p_list = soup.find_all('p')
    
    for elem in p_list:
        elem = " ".join(elem.text.lower().split())
        if not summary_flag:
            head = re.search(sum_start + "(.*)" + sum_end , elem )
            if head != None:
                head_str = head.group(1)
                summary_flag = True
                
        if summary_flag & (not full_flag):
            civil = re.search(full_start + "(.*)" + full_end , elem )
            if civil != None:
                civil_str = civil.group(1)
                full_flag = True
                break;
    if full_flag:
        df_obj = pd.DataFrame({"Summary":[head_str],"Full":[civil_str],"Doc":[filename]})
    else:
        return [] 
    return df_obj

In [0]:
df = pd.DataFrame()

directory = "/Users/kowshikmoyya/Documents/PGDBA/IIT Semester/IR/Project/Web_Pages/"
i = 0
for filename in os.listdir(directory):
    i = i + 1
    if i % 500 == 0:
        print(i)
    if filename.endswith(".html"):
        fname = os.path.join(directory,filename)
        f = open(fname)
        soup = BeautifulSoup(f,'html.parser')
        df_obj = extract2df(soup,filename)
        if len(df_obj) == 0:
            f.close()
            continue
        df = df.append(df_obj)
        f.close()
df_train = df.reset_index().drop(["index"],axis =1)
print(len(df))
print(len(df_train))

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
4946
4946


### Save the extracted dataframe

In [0]:
df_train.to_pickle("/Users/kowshikmoyya/Documents/PGDBA/IIT Semester/IR/Project/nf/data.pkl")

In [0]:
df_train.head(5)

,Summary,Full,Doc
0,proceedings under section 145(1) of the crimi...,civil appeal nos. 587-696 & 598-600 of 1976. ...,1976_299.html
1,when a tenant has neither paid nor tendered t...,civil appeal no. 966 of 1976. (appeal by spec...,1982_4.html
2,by a writ petition under article 226 of the c...,civil appeal no. 1654 of1967. appeal from the...,1968_205.html
3,the appellant is the mahant of emhar math of ...,civil appeal no. 1 770 of 1972. appeal by spe...,1978_167.html
4,the respondent assessee maintains accounts re...,civil appeal nos. 894-896 of 1971. from the j...,1976_49.html
